In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import string
import json
import re
from sklearn.utils import shuffle
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import sklearn
from sklearn.metrics import confusion_matrix
####
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model

from sklearn.linear_model import LogisticRegression
import sklearn.linear_model as linear_model
import sklearn.model_selection as model_selection

from sklearn import svm
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

In [2]:
# open training dataset and parse recipes

with open('train.json') as cooking_file:  
    
    data = json.load(cooking_file)
    

In [3]:
#COUNTVECTORIZER
#removing punctuations and spaces before fixing the data and keep them all in a dict  (italian, mexican)

punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

def gather_recipes(recipes, cuisine_country):
    
    data_list = []
    
    for _object in recipes:
        
        if _object.get('cuisine') == cuisine_country:
            
            ingredients = _object.get('ingredients') 

            _id = _object.get('id')
            
            for i in ingredients:
                
                _dict = dict()
                ingr = i.replace(' ', '').lower()
                    
                for ch in ingr: 
                    
                    if ch in punctuations: 
                        
                        ingr = ingr.replace(ch, '')  
                
                _dict['ingredients'] = ingr                
                _dict['id'] = _id
                data_list.append(_dict)
                
    return data_list


clean_italian_cuisine_data = gather_recipes(data, 'italian')
clean_mexican_cuisine_data = gather_recipes(data, 'mexican')

clean_filipino_cuisine_data = gather_recipes(data, 'filipino')
clean_british_cuisine_data = gather_recipes(data, 'british')
clean_moroccan_cuisine_data = gather_recipes(data, 'moroccan')

In [4]:
#creating dfs  (italian, mexican)

italian_df = DataFrame(clean_italian_cuisine_data)

mexican_df = DataFrame(clean_mexican_cuisine_data)


#creating dfs (filipino, british, moroccan )

filipino_df = DataFrame(clean_filipino_cuisine_data) 

british_df = DataFrame(clean_british_cuisine_data)

moroccan_df = DataFrame(clean_moroccan_cuisine_data)


In [5]:
#fixing data for CountVectorizer (italian, mexican)

it_unique_ids = []
it_ingredients = []
it_grouped = italian_df.groupby('id')

for ids, ing in it_grouped:
    
    it_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    it_ingredients.append(row)

    
mex_unique_ids = []
mex_ingredients = []
mex_grouped = mexican_df.groupby('id')

for ids, ing in mex_grouped:
    
    mex_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    mex_ingredients.append(row)
    
#fixing data for CountVectorizer (moroccan, filipino, british)

moroccan_unique_ids = []
moroccan_ingredients = []

moroccan_grouped = moroccan_df.groupby('id')

for ids, ing in moroccan_grouped:
    
    moroccan_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    moroccan_ingredients.append(row)


filipino_unique_ids = []
filipino_ingredients = []

filipino_grouped = filipino_df.groupby('id')

for ids, ing in filipino_grouped:
    
    filipino_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    filipino_ingredients.append(row)


british_unique_ids = []
british_ingredients = []

british_grouped = british_df.groupby('id')

for ids, ing in british_grouped:
    
    british_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    british_ingredients.append(row)

In [6]:
#using CountVectorizer  (italian, mexican) and adding one column Cuisine

it_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
it_array = it_vectorizer.fit_transform(it_ingredients)
it_array = it_array.toarray()
it_df = DataFrame(it_array, columns = it_vectorizer.get_feature_names(), index = it_unique_ids)
it_df['Cuisine'] = 0 #italian
it_df.to_csv('it_cleandata.csv', index_label = 'ID')

mex_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
mex_array = mex_vectorizer.fit_transform(mex_ingredients)
mex_array = mex_array.toarray()
mex_df = DataFrame(mex_array, columns = mex_vectorizer.get_feature_names(), index = mex_unique_ids)
mex_df['Cuisine'] = 1 #mexican
mex_df.to_csv('mex_cleandata.csv', index_label = 'ID')

#using CountVectorizer (moroccan, filipino, british)

moroccan_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
moroccan_array = moroccan_vectorizer.fit_transform(moroccan_ingredients)
moroccan_array = moroccan_array.toarray()
moroccan_df = DataFrame(moroccan_array, columns = moroccan_vectorizer.get_feature_names(), index = moroccan_unique_ids)
moroccan_df['Cuisine'] = 0 #moroccan
moroccan_df.to_csv('moroccan_cleandata.csv', index_label = 'ID')

british_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
british_array = british_vectorizer.fit_transform(british_ingredients)
british_array = british_array.toarray()
british_df = DataFrame(british_array, columns = british_vectorizer.get_feature_names(), index = british_unique_ids)
british_df['Cuisine'] = 1 #british
british_df.to_csv('british_cleandata.csv', index_label = 'ID')

filipino_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
filipino_array = filipino_vectorizer.fit_transform(filipino_ingredients)
filipino_array = filipino_array.toarray()
filipino_df = DataFrame(filipino_array, columns = filipino_vectorizer.get_feature_names(), index = filipino_unique_ids)
filipino_df['Cuisine'] = 2 #filipino
filipino_df.to_csv('filipino_cleandata.csv', index_label = 'ID')

In [7]:
#putting mexican and italian in one DataFrame and shuffle

df = pd.DataFrame(it_df)
df = df.append(mex_df, sort = False)
df.fillna(0, inplace = True)
df = df.astype('int64')

df = shuffle(df)

#putting moroccan, filipino, british in one DataFrame and shuffle

df_ = pd.DataFrame(moroccan_df)
df_ = df_.append(filipino_df, sort = False)
df_ = df_.append(british_df, sort = False)
df_.fillna(0, inplace = True)
df_ = df_.astype('int64')

df_ = shuffle(df_)

In [8]:
#TFIDFVECTORIZER
#removing punctuations before fixing the data and keep them all in a dict  (italian, mexican)

def gather_recipes(recipes, cuisine_country):
    
    data_list = []
    
    for _object in recipes:
        
        if _object.get('cuisine') == cuisine_country:
            
            ingredients = _object.get('ingredients') 
            
            _id = _object.get('id')
            
            for i in ingredients:
                
                _dict = dict()
                ingr = i.lower()
                    
                for ch in ingr: 
                    
                    if ch in punctuations: 
                        
                        ingr = ingr.replace(ch, '')  
                
                _dict['ingredients'] = ingr                
                _dict['id'] = _id
                data_list.append(_dict)
                
    return data_list

clean2_italian_cuisine_data = gather_recipes(data, 'italian')
clean2_mexican_cuisine_data = gather_recipes(data, 'mexican')

In [9]:
#creating dfs2  (italian, mexican)

italian2_df = DataFrame(clean2_italian_cuisine_data)

mexican2_df = DataFrame(clean2_mexican_cuisine_data)

In [10]:
#fixing data for TfidfVectorizer (italian, mexican)

it2_unique_ids = []
it2_ingredients = []
it2_grouped = italian2_df.groupby('id')

for ids, ing in it2_grouped:
    
    it2_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    it2_ingredients.append(row)

    
mex2_unique_ids = []
mex2_ingredients = []
mex2_grouped = mexican2_df.groupby('id')

for ids, ing in mex2_grouped:
    
    mex2_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    mex2_ingredients.append(row)


In [11]:
#using TfidfVectorizer  (italian, mexican) and adding one column Cuisine

it2_vectorizer = TfidfVectorizer(analyzer = 'word', binary = True)
it2_array = it2_vectorizer.fit_transform(it2_ingredients)
it2_array = it2_array.toarray()
it2_df = DataFrame(it2_array, columns = it2_vectorizer.get_feature_names(), index = it2_unique_ids)
it2_df['Cuisine'] = 0 #italian
it2_df.to_csv('it2_cleandata.csv', index_label = 'ID')

mex2_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
mex2_array = mex2_vectorizer.fit_transform(mex2_ingredients)
mex2_array = mex2_array.toarray()
mex2_df = DataFrame(mex2_array, columns = mex2_vectorizer.get_feature_names(), index = mex2_unique_ids)
mex2_df['Cuisine'] = 1 #mexican
mex2_df.to_csv('mex2_cleandata.csv', index_label = 'ID')

In [12]:
#putting mexican and italian in one DataFrame and shuffle

df2 = pd.DataFrame(it2_df)
df2 = df2.append(mex2_df, sort = False)
df2.fillna(0, inplace = True)

df2 = shuffle(df2)

In [13]:
# Multiple Logistic Regression 
lr_m = linear_model.LogisticRegression()

# Support Vector Machines
svm_m = svm.LinearSVC(random_state=0)
#Decision Trees
dtree = tree.DecisionTreeClassifier()

#k-NN
knn = KNeighborsClassifier(n_neighbors = 3)

# Naive Bayes
gnb = BernoulliNB(binarize = None)

In [14]:
print('************ CountVectorizer ITALIAN-MEXICAN ************')
X = df.loc[:, df.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df['Cuisine'] #just the ids in a list

************ CountVectorizer ITALIAN-MEXICAN ************
(14276, 3976)


- 1. ITALIAN MEXICAN CUISINES CLASSIFIERS (COUNTVECTORIZER)

In [15]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Logistic Regression mean scores****

scores: {'fit_time': array([2.29156423, 2.07139039, 2.36748147, 2.15417075, 2.23291612]), 'score_time': array([0.03142667, 0.02707291, 0.02686501, 0.02746511, 0.02678108]), 'test_accuracy': array([0.96393557, 0.96812609, 0.97127846, 0.96847636, 0.96847636]), 'test_precision_weighted': array([0.96425502, 0.96821594, 0.97133204, 0.96849138, 0.96890994]), 'test_recall_weighted': array([0.96393557, 0.96812609, 0.97127846, 0.96847636, 0.96847636]), 'test_f1_weighted': array([0.96387337, 0.96809583, 0.97125701, 0.96846032, 0.96841485])}

mean score test accuracy: 0.9680585684361311

mean score test precision weighted: 0.9682408628025702

mean score test recall weighted: 0.9680585684361311

mean score test f1-measure weighted: 0.9680202761171023
Confusion Matrix: [[7685  153]
 [ 303 6135]]


In [16]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(svm_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****SVM mean scores****

scores: {'fit_time': array([0.5015223 , 0.49257731, 0.49180746, 0.49814582, 0.49015641]), 'score_time': array([0.02652001, 0.02725959, 0.02665591, 0.02702188, 0.02673507]), 'test_accuracy': array([0.95973389, 0.96112084, 0.96567426, 0.96392294, 0.96357268]), 'test_precision_weighted': array([0.95980666, 0.96111301, 0.96567996, 0.96395163, 0.96378088]), 'test_recall_weighted': array([0.95973389, 0.96112084, 0.96567426, 0.96392294, 0.96357268]), 'test_f1_weighted': array([0.95969582, 0.96111395, 0.96567663, 0.96393145, 0.96352138])}

mean score test accuracy: 0.9628049223191905

mean score test precision weighted: 0.9628664288717828

mean score test recall weighted: 0.9628049223191905

mean score test f1-measure weighted: 0.9627878439172777
Confusion Matrix: [[7601  237]
 [ 294 6144]]


In [17]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****Decision-Tree mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(dtree, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Decision-Tree mean scores****

scores: {'fit_time': array([15.12672782, 16.10482025, 10.71734643, 11.17155337, 17.12565184]), 'score_time': array([0.02980947, 0.02962708, 0.0295558 , 0.02984643, 0.02960825]), 'test_accuracy': array([0.93627451, 0.93835377, 0.94535902, 0.94290718, 0.94010508]), 'test_precision_weighted': array([0.93637456, 0.93836322, 0.94541905, 0.943061  , 0.94010944]), 'test_recall_weighted': array([0.93627451, 0.93835377, 0.94535902, 0.94290718, 0.94010508]), 'test_f1_weighted': array([0.93619008, 0.93835803, 0.94537735, 0.94294207, 0.94010714])}

mean score test accuracy: 0.9405999107173517

mean score test precision weighted: 0.9406654533007677

mean score test recall weighted: 0.9405999107173517

mean score test f1-measure weighted: 0.9405949343010332
Confusion Matrix: [[7415  423]
 [ 401 6037]]


In [18]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(knn, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****k-NN mean scores****

scores: {'fit_time': array([4.32930851, 4.31445169, 4.23870182, 4.22338605, 4.40475249]), 'score_time': array([174.51442051, 173.4572897 , 173.73018026, 176.06029773,
       172.05108428]), 'test_accuracy': array([0.87289916, 0.87845884, 0.88966725, 0.88896673, 0.88336252]), 'test_precision_weighted': array([0.87587463, 0.87861395, 0.89018412, 0.88925118, 0.88406032]), 'test_recall_weighted': array([0.87289916, 0.87845884, 0.88966725, 0.88896673, 0.88336252]), 'test_f1_weighted': array([0.87186141, 0.87815542, 0.88930013, 0.88866895, 0.88291247])}

mean score test accuracy: 0.8826709002339991

mean score test precision weighted: 0.8835968397201324

mean score test recall weighted: 0.8826709002339991

mean score test f1-measure weighted: 0.8821796774904431
Confusion Matrix: [[7217  621]
 [1054 5384]]


In [19]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(gnb, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Naive-Bayes mean scores****

scores: {'fit_time': array([0.4727087 , 0.47220039, 0.48455167, 0.49316597, 0.4712646 ]), 'score_time': array([0.07191467, 0.02977133, 0.03066683, 0.03223658, 0.02977085]), 'test_accuracy': array([0.96253501, 0.96532399, 0.96882662, 0.9642732 , 0.96322242]), 'test_precision_weighted': array([0.9631368 , 0.96564877, 0.96909701, 0.96443051, 0.9639813 ]), 'test_recall_weighted': array([0.96253501, 0.96532399, 0.96882662, 0.9642732 , 0.96322242]), 'test_f1_weighted': array([0.96244554, 0.96526395, 0.96877828, 0.96422906, 0.96312379])}

mean score test accuracy: 0.9648362497363218

mean score test precision weighted: 0.9652588808943285

mean score test recall weighted: 0.9648362497363218

mean score test f1-measure weighted: 0.9647681236527653
Confusion Matrix: [[7704  134]
 [ 368 6070]]


In [20]:
print('************ TfidfVectorizer ITALIAN-MEXICAN ************')
X = df2.loc[:, df2.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df2['Cuisine'] #just the ids in a list

************ TfidfVectorizer ITALIAN-MEXICAN ************
(14276, 2006)


- 2. ITALIAN MEXICAN CUISINES CLASSIFIERS (TFIDFVECTORIZER)

In [21]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Logistic Regression mean scores****

scores: {'fit_time': array([0.50387096, 0.54896426, 0.53869247, 0.50846815, 0.51512122]), 'score_time': array([0.00981379, 0.01034141, 0.00992084, 0.00979733, 0.01001906]), 'test_accuracy': array([0.99439776, 0.99194396, 0.99509632, 0.99404553, 0.99264448]), 'test_precision_weighted': array([0.99445435, 0.99206042, 0.99513972, 0.99410944, 0.99274175]), 'test_recall_weighted': array([0.99439776, 0.99194396, 0.99509632, 0.99404553, 0.99264448]), 'test_f1_weighted': array([0.99439447, 0.99193697, 0.99509381, 0.99404182, 0.99263873])}

mean score test accuracy: 0.9936256113653869

mean score test precision weighted: 0.9937011354262015

mean score test recall weighted: 0.9936256113653869

mean score test f1-measure weighted: 0.9936211605960459
Confusion Matrix: [[7838    0]
 [  91 6347]]


In [22]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(svm_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****SVM mean scores****

scores: {'fit_time': array([0.22816968, 0.19442344, 0.19218183, 0.19507813, 0.19385576]), 'score_time': array([0.01025224, 0.01001906, 0.00987625, 0.01013851, 0.0099833 ]), 'test_accuracy': array([0.99579832, 0.99544658, 0.99649737, 0.99754816, 0.99509632]), 'test_precision_weighted': array([0.99583023, 0.99548403, 0.99651957, 0.99755348, 0.99513975]), 'test_recall_weighted': array([0.99579832, 0.99544658, 0.99649737, 0.99754816, 0.99509632]), 'test_f1_weighted': array([0.9957965 , 0.99544443, 0.99649611, 0.99754773, 0.99509383])}

mean score test accuracy: 0.9960773521317459

mean score test precision weighted: 0.9961054104209683

mean score test recall weighted: 0.9960773521317459

mean score test f1-measure weighted: 0.9960757179194314
Confusion Matrix: [[7837    1]
 [  55 6383]]


In [23]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****Decision-Tree mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(dtree, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Decision-Tree mean scores****

scores: {'fit_time': array([3.03472066, 2.88156915, 3.06712699, 3.23401332, 3.29732966]), 'score_time': array([0.01471829, 0.01543117, 0.01462579, 0.015167  , 0.01536655]), 'test_accuracy': array([0.99929972, 0.99824869, 0.99894921, 0.99859895, 0.99859895]), 'test_precision_weighted': array([0.99929972, 0.99825425, 0.99895122, 0.99860252, 0.99860252]), 'test_recall_weighted': array([0.99929972, 0.99824869, 0.99894921, 0.99859895, 0.99859895]), 'test_f1_weighted': array([0.99929972, 0.99824838, 0.9989491 , 0.99859875, 0.99859875])}

mean score test accuracy: 0.9987391033471182

mean score test precision weighted: 0.9987420449390234

mean score test recall weighted: 0.9987391033471182

mean score test f1-measure weighted: 0.9987389404856752
Confusion Matrix: [[7837    1]
 [  20 6418]]


In [24]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(knn, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****k-NN mean scores****

scores: {'fit_time': array([2.29190803, 2.29224062, 2.3801589 , 2.34800649, 2.33663702]), 'score_time': array([88.62505531, 87.49232769, 86.65746641, 87.59152246, 88.43015361]), 'test_accuracy': array([0.93837535, 0.93134851, 0.92924694, 0.94010508, 0.93064799]), 'test_precision_weighted': array([0.94459435, 0.93897645, 0.93732158, 0.94599808, 0.93842799]), 'test_recall_weighted': array([0.93837535, 0.93134851, 0.92924694, 0.94010508, 0.93064799]), 'test_f1_weighted': array([0.93775315, 0.93053847, 0.92837508, 0.93952523, 0.92982133])}

mean score test accuracy: 0.9339447723047188

mean score test precision weighted: 0.9410636894998191

mean score test recall weighted: 0.9339447723047188

mean score test f1-measure weighted: 0.9332026527743507
Confusion Matrix: [[7838    0]
 [ 943 5495]]


In [25]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(gnb, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Naive-Bayes mean scores****
scores: {'fit_time': array([0.11181951, 0.0980413 , 0.09703803, 0.09573936, 0.09495687]), 'score_time': array([0.01399779, 0.01077127, 0.01059604, 0.01069427, 0.01060367]), 'test_accuracy': array([0.97759104, 0.9737303 , 0.98108581, 0.97793345, 0.97232925]), 'test_precision_weighted': array([0.97846982, 0.97492946, 0.98171551, 0.97878633, 0.97365731]), 'test_recall_weighted': array([0.97759104, 0.9737303 , 0.98108581, 0.97793345, 0.97232925]), 'test_f1_weighted': array([0.97752966, 0.97364289, 0.98104325, 0.97787427, 0.9722317 ])}

mean score test accuracy: 0.9765339691042791

mean score test precision weighted: 0.9775116858369163

mean score test recall weighted: 0.9765339691042791

mean score test f1-measure weighted: 0.9764643549157632
Confusion Matrix: [[7838    0]
 [ 335 6103]]


- 3. MOROCCAN FILIPINO BRITISH CUISINES CLASSIFIERS (COUNTVECTORIZER)

In [26]:
print('********* CountVectorizer MOROCCAN, FILIPINO, BRITISH *********')
X = df_.loc[:, df_.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df_['Cuisine'] #just the ids in a list

********* CountVectorizer MOROCCAN, FILIPINO, BRITISH *********
(2380, 2074)


In [27]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Logistic Regression mean scores****

scores: {'fit_time': array([0.29624867, 0.25796819, 0.25993776, 0.25109363, 0.24553299]), 'score_time': array([0.00495696, 0.00488305, 0.00487161, 0.0048871 , 0.00487208]), 'test_accuracy': array([0.93487395, 0.94327731, 0.93907563, 0.91596639, 0.91806723]), 'test_precision_weighted': array([0.93541173, 0.94365042, 0.93905144, 0.91740126, 0.92004047]), 'test_recall_weighted': array([0.93487395, 0.94327731, 0.93907563, 0.91596639, 0.91806723]), 'test_f1_weighted': array([0.93482409, 0.943344  , 0.93905722, 0.91617617, 0.91835951])}

mean score test accuracy: 0.930252100840336

mean score test precision weighted: 0.9311110649318362

mean score test recall weighted: 0.930252100840336

mean score test f1-measure weighted: 0.9303521992152968
Confusion Matrix: [[774  34  13]
 [ 27 749  28]
 [ 14  50 691]]


In [28]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(svm_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****SVM mean scores****

scores: {'fit_time': array([0.05558324, 0.05702448, 0.05542731, 0.05629277, 0.0541172 ]), 'score_time': array([0.00491714, 0.00492215, 0.0049572 , 0.00487733, 0.00486684]), 'test_accuracy': array([0.94537815, 0.93697479, 0.93487395, 0.91386555, 0.92647059]), 'test_precision_weighted': array([0.94560793, 0.93747454, 0.93480219, 0.91451573, 0.92676778]), 'test_recall_weighted': array([0.94537815, 0.93697479, 0.93487395, 0.91386555, 0.92647059]), 'test_f1_weighted': array([0.9454194 , 0.93712818, 0.93461957, 0.91394462, 0.9265474 ])}

mean score test accuracy: 0.9315126050420167

mean score test precision weighted: 0.9318336338131683

mean score test recall weighted: 0.9315126050420167

mean score test f1-measure weighted: 0.9315318350559185
Confusion Matrix: [[781  28  12]
 [ 26 741  37]
 [ 14  46 695]]


In [29]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****Decision Tree mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(dtree, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Decision Tree mean scores****

scores: {'fit_time': array([0.18114734, 0.15859199, 0.14378667, 0.15273213, 0.14982343]), 'score_time': array([0.00514793, 0.00492597, 0.00511861, 0.00514579, 0.0050323 ]), 'test_accuracy': array([0.84033613, 0.81722689, 0.82773109, 0.82352941, 0.77941176]), 'test_precision_weighted': array([0.84168924, 0.81676293, 0.83044333, 0.82373102, 0.78370513]), 'test_recall_weighted': array([0.84033613, 0.81722689, 0.82773109, 0.82352941, 0.77941176]), 'test_f1_weighted': array([0.84079345, 0.81676191, 0.82832539, 0.82354009, 0.78063765])}

mean score test accuracy: 0.8176470588235294

mean score test precision weighted: 0.8192663317138272

mean score test recall weighted: 0.8176470588235294

mean score test f1-measure weighted: 0.818011699282092
Confusion Matrix: [[695  77  49]
 [ 75 631  98]
 [ 31 100 624]]


In [30]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(knn, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****k-NN mean scores****

scores: {'fit_time': array([0.23262882, 0.23002625, 0.22994304, 0.22854257, 0.22976184]), 'score_time': array([2.30292201, 2.3095541 , 2.30891156, 2.31486344, 2.30675411]), 'test_accuracy': array([0.82142857, 0.80462185, 0.79621849, 0.77941176, 0.79621849]), 'test_precision_weighted': array([0.82214236, 0.80760987, 0.79812981, 0.78271031, 0.79829613]), 'test_recall_weighted': array([0.82142857, 0.80462185, 0.79621849, 0.77941176, 0.79621849]), 'test_f1_weighted': array([0.82154988, 0.80388459, 0.79637339, 0.77882022, 0.79640466])}

mean score test accuracy: 0.799579831932773

mean score test precision weighted: 0.8017776971681257

mean score test recall weighted: 0.799579831932773

mean score test f1-measure weighted: 0.7994065502172664
Confusion Matrix: [[651  61 109]
 [ 86 626  92]
 [ 43  86 626]]


In [31]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(gnb, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Naive-Bayes mean scores****

scores: {'fit_time': array([0.04368877, 0.04318047, 0.04317832, 0.04296303, 0.0437057 ]), 'score_time': array([0.00542307, 0.00508142, 0.00545073, 0.00511765, 0.00572777]), 'test_accuracy': array([0.90966387, 0.92226891, 0.92647059, 0.89705882, 0.9012605 ]), 'test_precision_weighted': array([0.9142671 , 0.92491801, 0.92943803, 0.90328119, 0.90795503]), 'test_recall_weighted': array([0.90966387, 0.92226891, 0.92647059, 0.89705882, 0.9012605 ]), 'test_f1_weighted': array([0.91043223, 0.92261714, 0.9268308 , 0.89808902, 0.90199312])}

mean score test accuracy: 0.911344537815126

mean score test precision weighted: 0.9159718744022938

mean score test recall weighted: 0.911344537815126

mean score test f1-measure weighted: 0.9119924610904245
Confusion Matrix: [[744  67  10]
 [ 20 759  25]
 [ 14  75 666]]
